### GPU access and Upload libraries:

In [1]:
#GPU:

import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import numpy as np
import tensorflow as tf

seed = 1

np.random.seed(1)
tf.random.set_seed(1)

2023-04-17 01:01:30.598215: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import torch 
torch.cuda.is_available()

True

In [3]:
torch.cuda.current_device()

0

In [4]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from transformers import pipeline, set_seed
import textwrap
from pprint import pprint
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

### Download the Dataset:

In [5]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

File ‘robert_frost.txt’ already there; not retrieving.



In [6]:
#To see what is in it:
!cat robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there
Had worn them really about the same,

And both that morning equally lay
In leaves no step had trodden black.
Oh, I kept the first for another day! 
Yet knowing how way leads on to way
I doubted if I should ever come back.

I shall be telling this with a sigh
Somewhere ages and ages hence:
Two roads diverged in a wood, and I,
I took the one less traveled by,
And that has made all the difference.

Whose woods these are I think I know.
His house is in the village, though; 
He will not see me stopping here
To watch his woods fill up with snow.

My little horse must think it queer
To stop without a farmhouse near
Between the woods and frozen lake
The darkest evenin

### Text Generator and data pre-processing:

In [7]:
gen = pipeline("text-generation", device=0) 
#uses GPT-2 
#'num_return_sequences=3' 
#'max_length=30'

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [24]:
#or: lst_lines=[line.rstrip() for line in open('robert_frost.txt')]
lst_lines=[]
for line in open('robert_frost.txt'):
    lst_lines.append(line.rstrip()) #or strip()
lst_lines = [line for line in lst_lines if len(line)>0]    

In [25]:
lst_lines

['Two roads diverged in a yellow wood,',
 'And sorry I could not travel both',
 'And be one traveler, long I stood',
 'And looked down one as far as I could',
 'To where it bent in the undergrowth;',
 'Then took the other, as just as fair,',
 'And having perhaps the better claim',
 'Because it was grassy and wanted wear,',
 'Though as for that the passing there',
 'Had worn them really about the same,',
 'And both that morning equally lay',
 'In leaves no step had trodden black.',
 'Oh, I kept the first for another day!',
 'Yet knowing how way leads on to way',
 'I doubted if I should ever come back.',
 'I shall be telling this with a sigh',
 'Somewhere ages and ages hence:',
 'Two roads diverged in a wood, and I,',
 'I took the one less traveled by,',
 'And that has made all the difference.',
 'Whose woods these are I think I know.',
 'His house is in the village, though;',
 'He will not see me stopping here',
 'To watch his woods fill up with snow.',
 'My little horse must think it q

In [26]:
set_seed(1234) #meh

In [28]:
lst_lines[0]

'Two roads diverged in a yellow wood,'

In [37]:
gen(lst_lines[0]) #can also add 'max_length=20' and 'num_return_sequences=3'

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Two roads diverged in a yellow wood, then I went to the point where my father and the driver were parked. At one point they were sitting in the rear of the car (a small car), and I got behind them and went for the'}]

In [38]:
pprint(gen(lst_lines[0])) #or pprint(_)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Two roads diverged in a yellow wood, only about 1.7 mi '
                    'from the base of Mt. Vesuvius in what remains of the '
                    'volcano. The road heads to the top of a mountain. Its '
                    'east side has three hills, all paved'}]


In [39]:
pprint(_)

[{'generated_text': 'Two roads diverged in a yellow wood, then I went to the '
                    'point where my father and the driver were parked. At one '
                    'point they were sitting in the rear of the car (a small '
                    'car), and I got behind them and went for the'}]


### Pretyinfthe prints and fiddling more with it so that every other line of teh poem is generated by GPT-2: (meh)

In [45]:
def wrap_lines(x):
    return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)

In [53]:
out = gen(lst_lines[0], max_length=30)
print(wrap_lines(out[0]['generated_text']))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Two roads diverged in a yellow wood, and there was a man who took a
right turn on to Hwy.  1 and said he was off


In [60]:
prev = 'Two roads diverged in a yellow wood, and there was a man who took a' + \
' right turn on to Hwy.' 
#deleting the last incomplete sentence 

out = gen(prev + '\n' + lst_lines[2], max_length=60)
print(wrap_lines(out[0]['generated_text']))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Two roads diverged in a yellow wood, and there was a man who took a
right turn on to Hwy.
And be one traveler, long I stood on the road
before the line's opening, and let a man come and drag thee out of the
field, and I gave thee my


In [61]:
prev = 'Two roads diverged in a yellow wood, and there was a man who took a' + \
' right turn on to Hwy.' + \
' And be one traveler, long I stood, and long we' + \
' did not walk.' + \
' That a stranger would draw his sword to the side, and' + \
' lead a man into him,'

out = gen(prev + '\n' + lst_lines[4], max_length=90)
print(wrap_lines(out[0]['generated_text']))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Two roads diverged in a yellow wood, and there was a man who took a
right turn on to Hwy.  And be one traveler, long I stood, and long we
did not walk.  That a stranger would draw his sword to the side, and
lead a man into him,
To where it bent in the undergrowth; where it had
never passed.  And his heart and his spirit had been at a state of
bliss, and he


### GPT was not trained on poetry, that's why the generated poems are not very coherent. But for facts and statements, it works much better. Example:

In [64]:
prompt = 'Neural networks with attention have been used with great success' + \
' in natural language processing.'

out = gen(prompt, max_length=300)
print(wrap_lines(out[0]['generated_text']))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Neural networks with attention have been used with great success in
natural language processing.

This video shows a recent effort to
demonstrate the work done by a group of researchers at the University
of Cambridge using three different neural networks: a neural network
called Akaike (the network that learns the shape of words), the
akaikitim (the network that processes a word), and a neural network
called Gluon (the network that learns to change the meaning of a
word). The first is a group of computer scientists from Cambridge.
The researchers looked specifically for a key factor influencing the
human brain: the strength of a human's connections to a network.

We
found it.

The neural networks, based on data from one of Google's
DeepMind deep learning teams, were trained on just 4- and 6-year-olds
and their brains were tuned for the three networks for a single second
using a technique called the Fourier transform.  To get the human
brain to learn to apply the technique, the research